- create .venv
- run `pip install -r requirements.txt`
- get API key from https://kisski.gwdg.de/leistungen/2-02-llm-service/
- create `.env` file with:
  > OPENAI_API_KEY = "YOUR-API-KEY"  
  > KISSKI_URL = "https://chat-ai.academiccloud.de/v1"

In [17]:
# 1. Load HTML File/s
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="html").load_data()
print(f"Loaded {len(documents)} documents.")

# 2. Chunk with SentenceSplitter
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200) # 1 token = 4 characters
nodes = splitter.get_nodes_from_documents(documents)
print(f"Generated {len(nodes)} chunks.") 


# 3. Embed Chunks with HuggingFace
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


Loaded 1 documents.
Generated 326 chunks.


In [18]:
# 4. Create Index
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex, StorageContext

faiss_index = faiss.IndexFlatL2(384)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes,
    embed_model=embed_model,
    storage_context=storage_context,
)

# Persist index to disk
index.storage_context.persist(persist_dir="./faiss_index")
print("Index stored in ./faiss_index")

Index stored in ./faiss_index


In [19]:
# 5. Define LLM

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("KISSKI_URL")

if not api_key or not base_url:
    raise ValueError("Missing API key or URL.")

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

def ask_openai_llm(prompt: str) -> str:
    response = client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [20]:
# 6. Ask a Question
import textwrap
from IPython.display import Markdown, display

while True:
    query = input("Enter your question (or type 'q'): ").strip()
    if query.lower() == 'q':
        print("Session ended.")
        break

    nodes = index.as_retriever().retrieve(query)
    context = "\n---\n".join([n.get_content() for n in nodes])
    full_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question:
{query}"""

    answer = ask_openai_llm(full_prompt)
    print(f"\nQ:")
    display(Markdown(textwrap.dedent(query)))
    print("\nA:")
    display(Markdown(textwrap.dedent(answer)))
    print("___\n")


Q:


How is this change impacting Native peoples?


A:


Unfortunately, the provided text does not specifically address the impact of climate change on Native peoples. The text primarily focuses on the Pacific Islands, including Samoa, Dominica, and Kiribati, and discusses the role of culture, social capital, and traditional knowledge in adaptation efforts.

However, it's worth noting that the text mentions that "culture is often overlooked in adaptation policies and plans" (referring to the National Communications of 16 Small Island Developing States). This suggests that the experiences and perspectives of indigenous peoples may be marginalized or ignored in climate change adaptation efforts.

If you're looking for information on the impact of climate change on Native peoples or indigenous communities, I'd be happy to try and provide more general information or point you in the direction of additional resources.

___

Session ended.


In [21]:
# for i, node in enumerate(nodes[:5]):
#     print(f"\nChunk {i + 1}")
#     print(f"Characters: {len(node.get_content())}")
#     print(f"Preview: {node.get_content()[:500]}...")